In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molmodmt as m3t

/home/diego/Myusr/opt/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/yank-0.24.2-py3.7-linux-x86_64.egg/yank/experiment.py:1170: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)


_ColormakerRegistry()

In [16]:
molsys = m3t.load('/home/diego/1brs.pdb')

AttributeError: 'fortran' object has no attribute 'lengths_and_angles_to_box'

In [10]:
from numpy import array

In [11]:
array([1,3], dtype='float64')

array([1., 3.])